# Success Rate Evaluation

---

This script will help us evaluate the quality of the generated packets, using the same evaluation method proposed on the paper PAC-GAN: Packet Generation of Network Traffic using Generative Adversarial Networks. This paper proposes the success rate as:

*Success rate as the number of packets that are successfully sent divided by the total number of packets generated by our GAN*.

## Step 1: Generate the Ethernet encapsulation
---

Most of the generated packets are generated down until IP level, in order to send them, we must first encapsulate them in Ethernet frames. For this frames we are going to use the user MAC as sender and a random MAC as receiver.

In [4]:
import uuid
import time

# joins elements of getnode() after each 2 digits.
usr_MAC =':'.join(['{:02x}'.format((uuid.getnode() >> ele) & 0xff)
for ele in range(0,8*6,8)][::-1])

print ("The user MAC address in formatted way is : " + usr_MAC)

The user MAC address in formatted way is : 00:41:0e:e7:83:cc


In [6]:
import random

rand_MAC= "02:00:00:%02x:%02x:%02x" % (random.randint(0, 255),
                             random.randint(0, 255),
                             random.randint(0, 255))
print ("The random generated MAC address in formatted way is : " + rand_MAC)


The random generated MAC address in formatted way is : 02:00:00:92:5a:06


***The above cell gives you a unicast MAC address that is 100% safe to use in your environment, and isn't trampling on anyone's registered MAC address space***

In [9]:
from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.l2 import Ether
from scapy.layers.inet import IP
from scapy.layers.http import *

eth = Ether(src=usr_MAC, dst=rand_MAC)
eth.show()

###[ Ethernet ]### 
  dst       = 02:00:00:92:5a:06
  src       = 00:41:0e:e7:83:cc
  type      = 0x9000



C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


## Step 2: Create functions to evaluate packets 

In [11]:
def has_eth_header(packet):
    return Ether in packet

In [12]:
from struct import unpack

def has_eth_header_Bin(packet):
    # Check if the packet length is at least 14 bytes
    if len(packet) < 14:
        return False

    # Unpack the first 14 bytes of the packet
    eth_header = unpack('!6s6sH', packet[:14])

    # Extract the EtherType field
    ether_type = eth_header[2]

    # Check if the EtherType field is within the range for Ethernet II framing
    if ether_type >= 0x0600:
        return True
    else:
        # This could be an IEEE 802.3 Ethernet frame
        return False

In [13]:
def Ether_packet_malformed(packet, verbose=True):
    
    if len(packet) < 14:
        return False
    
    if packet[Ether].type < 0x0600:
        return "Wrong Ether type"
    
    if packet.haslayer(ARP):
        var= ARP_packet_malformed(packet)
    else:
        var = False
        
    if var != False:
        return var
    
    return False

In [51]:
def ip_packet_malformed(packet, verbose=True):
    
    # Minimum length check for IP header
    if len(packet) < 20:
        if verbose ==True:
            print("\tPacket too short: ")
        return "IP error"  # Packet is too short to be a valid IP packet
    
    # Version should be 4 for IPv4
    if packet[IP].version != 4:
        return "IP error"  # Invalid version

    # IHL (Internet Header Length) should be at least 5 for a valid IP header without options
    if packet[IP].ihl < 5:
        if verbose ==True:
            print("\tPacket ihl too short: ")
        return "IP error"  # Invalid IHL

    # Total length field should match the actual packet length
    if packet[IP].len != len(packet):
        if verbose ==True:
            print("\tLength doesnt match: ")
        return "IP error"  # Mismatched total length

    # Protocol field should indicate ICMP (1), HTTP (6 for TCP) or DNS (17 for UDP)
    if packet[IP].proto not in (1,6, 17):
        if verbose ==True:
            print("\tWrong protocol: ")
        return "IP error"  # Unsupported protocol for this check

    # If all checks pass, the IP packet is not malformed, then we have to evaluate specific protocol malformation
    
    if packet[IP].proto == 1: #ICMP
        var = icmp_packet_malformed(packet[ICMP], verbose)
        if var != False:
            return var
        
    elif packet[IP].proto == 17: #UDP -> DNS
        var = UDP_packet_malformed(packet[UDP], verbose)
        if var != False:
            return var    
        var = DNS_packet_malformed(packet, verbose)
        if var != False:
            return var
    
    elif packet[IP].proto == 6: #TCP -> HTTP
        var = TCP_packet_malformed(packet[TCP], verbose)
        if var != False:
            return var    
        var = HTTP_packet_malformed(packet[TCP], True)
        if var != False:
            return var
        
    return False

In [53]:
def icmp_packet_malformed(packet, verbose=True):
    
    if packet.haslayer(ICMP) == False:
        if verbose ==True:
            print("\tICMP: ICMP Packet non-existent")
        return  "DNS non-existent"  # Packet is too short to be a valid DNS packet
    
     # Minimum length check for ICMP header
    if len(packet) < 8:
        return  "ICMP malformation"  # Packet is too short to be a valid ICMP packet

    # Check correct ICMP type
    if packet[ICMP].type not in (0, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 40, 41, 42, 43):
        if verbose ==True:
            print("\tIncorrect ICMP type")
        return "ICMP malformation"
    
    #Check correct ICMP code acording to type
    if packet[ICMP].type == 0 and packet[ICMP].code != 0: # Echo-Reply
        if verbose ==True:
            print("\tIncorrect ICMP code for type 0 (Echo-Reply)")
        return "ICMP malformation"
    if packet[ICMP].type == 3 and packet[ICMP].code not in range(0,16): # Destination Unreachable
        if verbose ==True:
            print("\tIncorrect ICMP code for type 3 (Destination Unreachable)")
        return "ICMP malformation"
    if packet[ICMP].type == 5 and packet[ICMP].code not in range(0,4): # Redirect
        if verbose ==True:
            print("\tIncorrect ICMP code for type 5 (Redirect)")
        return "ICMP malformation"
    if packet[ICMP].type == 8 and packet[ICMP].code != 0: # Echo
        if verbose ==True:
            print("\tIncorrect ICMP code for type 8 (Echo)")
        return "ICMP malformation"
    if packet[ICMP].type == 9 and packet[ICMP].code not in (0,16): # Router Advertisement
        if verbose ==True:
            print("\tIncorrect ICMP code for type 9 (Router Advertisement)")
        return "ICMP malformation"
    if packet[ICMP].type == 10 and packet[ICMP].code != 0: # Router Selection
        if verbose ==True:
            print("\tIncorrect ICMP code for type 10 (Router Selection)")
        return "ICMP malformation"
    if packet[ICMP].type == 11 and packet[ICMP].code not in (0,1): # Time Exceeded
        if verbose ==True:
            print("\tIncorrect ICMP code for type 11 (Time Exceeded)")
        return "ICMP malformation"
    if packet[ICMP].type == 12 and packet[ICMP].code not in (0,1, 2): # Parameter Problem
        if verbose ==True:
            print("\tIncorrect ICMP code for type 12 (Parameter Problem)")
        return "ICMP malformation"
    if packet[ICMP].type == 13 and packet[ICMP].code != 0: # Timestamp
        if verbose ==True:
            print("\tIncorrect ICMP code for type 13 (Timestamp)")
        return "ICMP malformation"
    if packet[ICMP].type == 14 and packet[ICMP].code != 0: # Timestamp Reply
        if verbose ==True:
            print("\tIncorrect ICMP code for type 14 (Timestamp Reply)")
        return "ICMP malformation"
    if packet[ICMP].type == 40 and packet[ICMP].code not in range(0, 6): # Photuris
        if verbose ==True:
            print("\tIncorrect ICMP code for type 40 (Photorius)")
        return "ICMP malformation"
    if packet[ICMP].type == 42 and packet[ICMP].code !=0 : # Extended Echo Request
        if verbose ==True:
            print("\tIncorrect ICMP code for type 42 (Extender Echo Request)")
        return "ICMP malformation"
    if packet[ICMP].type == 43 and packet[ICMP].code not in range (0, 6) : # Extended Echo Reply
        if verbose ==True:
            print("\tIncorrect ICMP code for type 43 (Extended Echo-Reply)")
        return "ICMP malformation"

    return False

In [55]:
def UDP_packet_malformed(packet, verbose=True):
    # Check if the packet is at least the size of a UDP header
    if len(packet) < 8:
        return  "UDP malformation"  # Packet is too short to be a valid UDP packet
    
    return False

In [57]:
def DNS_packet_malformed(packet, verbose=True):

    if packet.haslayer(DNS) == False:
        if verbose ==True:
            print("\tDNS: DNS Packet non-existent")
        return  "DNS non-existent"  # Packet is too short to be a valid DNS packet
    
        # Check if the packet is at least the size of a DNS header
    if len(packet) < 12:
        if verbose ==True:
            print("\tDNS: Packet too short")
        return  "DNS malformation"  # Packet is too short to be a valid DNS packet
    
    if packet[DNS].qr not in (0,1): #Check QR field
        if verbose ==True:
            print("\tDNS: Wrong QR field")
        return  "DNS malformation"
    if packet[DNS].opcode not in (0,1, 2): #Check opcode field
        if verbose ==True:
            print("\tDNS: Wrong opcode field")
        return  "DNS malformation"
    if packet[DNS].aa not in (0,1): #Check AA variable
        if verbose ==True:
            print("\tDNS: Wrong AA field")
        return  "DNS malformation"
    if packet[DNS].tc not in (0,1): #Check TC variable
        if verbose ==True:
            print("\tDNS: Wrong TC field")
        return  "DNS malformation"
    if packet[DNS].rd not in (0,1): #Check RD variable
        if verbose ==True:
            print("\tDNS: Wrong RD field")
        return  "DNS malformation"
    if packet[DNS].ra not in (0,1): #Check RA variable
        if verbose ==True:
            print("\tDNS: Wrong RA field")
        return  "DNS malformation"
    if packet[DNS].rcode not in (0,1, 2, 3): #Check RCODE variable
        if verbose ==True:
            print("\tDNS: Wrong RCODE field")
        return  "DNS malformation"
    
    ## Check if the number of queries and responses is correct ##
    
    # Queries
    count = 0
    # Iterate over the DNSQR layer
    current_layer = packet[DNS].qd
    while current_layer:
        count += 1
        current_layer = current_layer.payload
    if packet[DNS].qdcount != count:
        if verbose ==True:
            print("\tDNS: qdcount and number of queries don't match")
            print("\t\tqdcount: " + str(packet[DNS].qdcount))
            print("\t\tQuestions detected: " + str(count))
        return  "DNS malformation"
    
    #Responses
    count = 0
    # Iterate over the an layer
    current_layer = packet[DNS].an
    while current_layer:
        count += 1
        current_layer = current_layer.payload
    if packet[DNS].ancount != count:
        if verbose ==True:
            print("\tDNS: ancount and number of responses don't match")
            print("\t\tancount: " + str(packet[DNS].ancount))
            print("\t\tResponses detected: " + str(count))
        return  "DNS malformation"
    
    #Aditional Resources
    count = 0
    # Iterate over the an layer
    current_layer = packet[DNS].ar
    while current_layer:
        count += 1
        current_layer = current_layer.payload
    if packet[DNS].arcount != count:
        if verbose ==True:
            print("\tDNS: arcount and number of aditional resources don't match")
            print("\t\tarcount: " + str(packet[DNS].arcount))
            print("\t\tAditional Resources detected: " + str(count))
        return  "DNS malformation"
    
    #NS
    count = 0
    # Iterate over the DNSNS layer
    current_layer = packet[DNS].ns
    while current_layer:
        count += 1
        current_layer = current_layer.payload
    if packet[DNS].nscount != count:
        if verbose ==True:
            print("\tDNS: nscount and number of NSs resources don't match")
            print("\t\tnscount: " + str(packet[DNS].arcount))
            print("\t\tAuthority Responses detected: " + str(count))
        return  "DNS malformation"

    return False

In [59]:
def ARP_packet_malformed(packet, verbose=True):

        # Check if the packet is at least the size of a DNS header
    if len(packet) < 28:
        if verbose ==True:
            print("\ARP: Packet too short")
        return  "ARP malformation"  # Packet is too short to be a valid ARP packet
    
    if packet[ARP].op not in (1,2):
        if verbose ==True:
            print("\ARP: Wrong opcode")
        return  "ARP malformation"  # Packet is too short to be a valid ARP packet
    
    return False

In [61]:
def TCP_packet_malformed(packet, verbose=True):

    if not packet.haslayer(TCP):
        if verbose ==True:
            print("\tTCP: TCP Packet non-existent")
        return "TCP non-existent"
    
    # Check if the packet is at least the size of a DNS header
    if len(packet) < 20:
        if verbose ==True:
            print("\tTCP: Packet too short")
        return  "TCP malformation"  # Packet is too short to be a valid DNS packet
    
    if packet[TCP].sport not in range(0,65535): #Check QR field
        if verbose ==True:
            print("\tTCP: Incorrect sender port number")
        return  "TCP malformation"
    if packet[TCP].dport not in range(0,65535): #Check QR field
        if verbose ==True:
            print("\tTCP: Incorrect destination port number")
        return  "TCP malformation"
    if packet[TCP].seq not in range(0,pow(2,32)): #Check QR field
        if verbose ==True:
            print("\tTCP: Incorrect seq number")
        return  "TCP malformation"
    if packet[TCP].ack not in range(0,pow(2,32)): #Check QR field
        if verbose ==True:
            print("\tTCP: Incorrect ack number")
        return  "TCP malformation"
    if packet[TCP].window not in range(0,65535): #Check QR field
        if verbose ==True:
            print("\tTCP: Incorrect Window size")
        return  "TCP malformation"
    if packet[TCP].urgptr not in range(0,65535): #Check QR field
        if verbose ==True:
            print("\tTCP: Incorrect urgent pointer")
        return  "TCP malformation"
    
    return False

In [63]:
def HTTP_packet_malformed(packet, verbose=True):

    # No es aplicable a conversaciones, dado que la mayoría de los paquetes de una conversación HTTP son de TCP handshake, ACKs o TCP teardown
    
    #if not packet.haslayer(HTTP):
    #    if verbose ==True:
    #        print("\tHTTP: HTTP Packet non-existent")
    #    return "HTTP non-existent"
    
    try:
        if packet[HTTP].Http_Version not in (b'HTTP/1.1', b'HTTP/1.0'):
            if verbose ==True:
                print("\tHTTP: HTTP Version isn´t HTTP 1.1 or HTTP 1.0")
            return "HTTP malformation"
    except:
        if verbose ==True:
            print("\tHTTP: Packet can´t be read")
        #return "HTTP malformation"
    
    if packet.haslayer(HTTPRequest):
        if packet[TCP].dport != 80:
            if verbose ==True:
                print("\tHTTP: Destination Port does not correspond to HTTP port (Packet is an HTTP Request)")
            return  "HTTP malformation"
        if packet[HTTP].Method not in (b'GET', b'OPTIONS', b'POST', b'HEAD', b'PUT', b'DELETE', b'TRACE', b'CONNECT'):
            if verbose ==True:
                print("\tHTTP: Wrong HTTP Request method.")
            return  "HTTP malformation"
    
    if packet.haslayer(HTTPResponse):
        if packet[TCP].sport != 80:
            if verbose ==True:
                print("\tHTTP: Source Port does not correspond to HTTP port (Packet is an HTTP Response)")
            return  "HTTP malformation"
        if packet[HTTP].Status_Code not in (b'100', b'101', b'102', b'200', b'201', b'202', b'203', b'204', b'205', b'206', b'207', b'208', b'226', b'300', b'301', b'302', b'303', b'304', b'305', b'306', b'307', b'308', b'400', b'401', b'402', b'403', b'404', b'405', b'406', b'407', b'408', b'409', b'410',b'411', b'412', b'413', b'414', b'415', b'416', b'417', b'418', b'421', b'422', b'423', b'424', b'426', b'428', b'429', b'431', b'444', b'451', b'499', b'500', b'501', b'502', b'503', b'504', b'505', b'506', b'507', b'508', b'510', b'511', b'599'):
            if verbose ==True:
                print("\tHTTP: Wrong HTTP Response status code.")
            return  "HTTP malformation"
        
    return False

## Step 3: Generate a function to send packets and evaluate the results
---
This function will be called every time a packet is read while processing the pcap. Firstly, we have to check if the packet already has the eth header or we need to add it.

We have different forms of sending the packet, these are explained in [Sending & recieving packets](https://0xbharath.github.io/art-of-packet-crafting-with-scapy/scapy/sending_recieving/index.html).

As some packets do have eth header and some doesn't, I decided to send them all the same way using sendp() (Layer2) so they are all equally treated.

In [66]:
def send_packet(packet, verbose =True):
    
    if packet.haslayer(Ether):
        var = Ether_packet_malformed(packet, verbose)
    if packet.haslayer(IP):
        var = ip_packet_malformed(packet, verbose)
    if var != False:
        if verbose == True:
            print("\tPacket is malformed, couldn't send it")
        return var
    
    #Check if the packet already has an eth header
    if has_eth_header(packet):
        if verbose == True:
            print("\tPacket already has an Ethernet header.")
        sendp(packet, verbose=False)
        if verbose == True:
            print ("\tPacket sent!")
        return "sent"
    else:
        sendp(eth/packet, verbose=False)
        if verbose == True:
            print ("\tPacket sent!")
        return "sent"
        

## Step 3: Load the generated packets from the pcaps
---
After created, the packets are automatically saved in their respective pcaps, so we only need lo load them and encapsulate them.

There are different forms of reading a pcap:

1. Using **rdpcap()**: This form loads the whole pcap into memory, making it somehow ineficient.
2. Using **PcapReader()**: This allows you to iterate over the packets captured in a file and parse them. The file is not completely loaded into memory. This is little bit more memory efficient.

In [80]:
## PACKETS
#file_name1 = "../data/Packets/ICMP/pcap/ICMPgenerated_3_5_turbo-instruct_conv.pcap"
#file_name2 = "../data/Packets/DNS/pcap/DNSgenerated_3_5_turbo-instruct_conv.pcap"
#file_name3 = "../data/Packets/ARP/pcap/Training_v2_ARPgenerated_3_5_turbo-instruct_conv.pcap"
#file_name4 = "../data/Packets/HTTP/pcap/HTTPgenerated_3_5_turbo-instruct_conv_ALL.pcap"

## CONVERSATIONS
#file_name1 = "../data/Conversations/ICMP/pcap/ICMP_v3_generated_3_5_turbo-instruct_conv.pcap"
#file_name2 = "../data/Conversations/DNS/pcap/DNS_v3_generated_3_5_turbo-instruct_conv.pcap"
#file_name3 = "../data/Conversations/ARP/pcap/ARP_v3_generated_3_5_turbo-instruct_conv.pcap"
#file_name4 = "../data/Conversations/HTTP/pcap/HTTP_Conv2_generated_3_5_turbo-instruct_conv_ALL.pcap"
#file_name5 = "../data/Conversations/TCP/pcap/TCPgenerated_3_5_turbo-instruct_conv_prev.pcap"

## FINE-TUNED
#file_name1 = "../data/Fine_Tuned/ICMP/pcap/Test_ICMP_CONV_v2_generated_3_5_turbo-instruct_conv.pcap"
#file_name2 = "../data/Fine_Tuned/DNS/pcap/Test_DNS_CONV_v2_generated_3_5_turbo-instruct_conv.pcap"
#file_name3 = "../data/Fine_Tuned/ARP/pcap/Test_ARP_CONV_v2_generated_3_5_turbo-instruct_conv.pcap"
#file_name4 = "../data/Fine_Tuned/HTTP/pcap/Test_HTTP_CONV_v2_generated_3_5_turbo-instruct_conv.pcap"

#MOE
file_name1 = "../data/MOE/pcap/MOE_Conversations_generated_3_5_turbo-instruct_conv.pcap"

In order to collect all the information about the origin of the error of each packet, I established a basic error retrieving system formed by messages passed trough the functions, ending that on a list where different error can be counted.

In [83]:
protocols = ["IP", "ICMP", "DNS", "UDP", "ARP", "HTTP", "TCP"]
protocol_count = [0] * len(protocols) 
errors = ["IP error", "ICMP malformation", "DNS malformation", "UDP malformation", "ARP malformation", "HTTP malformation", "TCP malformation"]
nonEx = ["IP non-existent", "ICMP non-existent", "DNS non-existent","UDP non-existent", "ARP non-existent", "HTTP non-existent", "TCP non-existent"]
arrNonEx = [0] * len(errors)
arrEr = [0] * len(errors)
total_count = 0

In [85]:
def process_pcap(file_name, verbose=True):
    print('Opening {}...'.format(file_name))
    
    packets=[]
    global total_count
    
    for packet,metadata in RawPcapReader(file_name):
        total_count += 1
        if verbose == True:
            print("Packet nº " +str(total_count) +"\t")
        
        if file_name in (file_name3):
            packet = Ether(packet)
        else:
            packet=IP(packet)
        packets.append(packet)
        
        if IP in packet:
            protocol_count[protocols.index("IP")] += 1
        if ICMP in packet:
            protocol_count[protocols.index("ICMP")] += 1
        if DNS in packet:
            protocol_count[protocols.index("DNS")] += 1 
        if UDP in packet:
            protocol_count[protocols.index("UDP")] += 1
        if ARP in packet:
            protocol_count[protocols.index("ARP")] += 1
        if HTTP in packet:
            protocol_count[protocols.index("HTTP")] += 1
        if TCP in packet:
            protocol_count[protocols.index("TCP")] += 1
    
        control = send_packet(packet, verbose)
        
        if control == "IP error":
            arrEr[errors.index(control)] += 1
        if control == "ICMP malformation":
            arrEr[errors.index(control)] += 1
        if control == "DNS malformation":
            arrEr[errors.index(control)] += 1
        if control == "UDP malformation":
            arrEr[errors.index(control)] += 1
        if control == "ARP malformation":
            arrEr[errors.index(control)] += 1
        if control == "HTTP malformation":
            arrEr[errors.index(control)] += 1
        if control == "TCP malformation":
            arrEr[errors.index(control)] += 1
            
        if control == "IP non-existent":
            arrNonEx[nonEx.index(control)] += 1
        if control == "ICMP non-existent":
            arrNonEx[nonEx.index(control)] += 1
        if control == "DNS non-existent":
            arrNonEx[nonEx.index(control)] += 1
        if control == "UDP non-existent":
            arrNonEx[nonEx.index(control)] += 1
        if control == "ARP non-existent":
            arrNonEx[nonEx.index(control)] += 1
        if control == "HTTP non-existent":
            arrNonEx[nonEx.index(control)] += 1
        if control == "TCP non-existent":
            arrNonEx[nonEx.index(control)] += 1

## Step 5: Send all the packets and retrieve errors info
---

In [88]:
arrEr = [0] * len(errors)
protocol_count = [0] * len(protocols) 

process_pcap(file_name1, verbose =False) #ICMP
#process_pcap(file_name2, verbose =False) #DNS
#process_pcap(file_name3, verbose =False) #ARP
#process_pcap(file_name4, verbose =False) #HTTP
#process_pcap(file_name5, verbose =False) #TCP

Opening ../data/MOE/pcap/MOE_Conversations_generated_3_5_turbo-instruct_conv.pcap...
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Destination Port does not correspond to HTTP port (Packet is an HTTP Request)
	HTTP: Packet can´t be read
	HTTP: Source Port does not correspond to HTTP port (Packet is an HTTP Response)
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t be read
	HTTP: Packet can´t b

## Step 4: Results Evaluation
---

In [91]:
 print("\n\n--------------------------------------- Fine-Tunning Results evaluation ---------------------------------------\n")
print("Protocol    nº detections    Correctly sent    Malformed    Non-existent    %SuccessRate    %presence in total")
for protocol in protocols:
    Cor_sent = protocol_count[protocols.index(protocol)]-arrEr[protocols.index(protocol)]
    malformed = arrEr[protocols.index(protocol)]
    nonExistent = arrNonEx[protocols.index(protocol)]
    SuccessRate = (Cor_sent/(Cor_sent + malformed + nonExistent))*100 if (Cor_sent + malformed + nonExistent) != 0 else None
    Presence = ((Cor_sent + malformed)/total_count)*100
    if SuccessRate != None:
        print(protocol, "\t\t" + str(protocol_count[protocols.index(protocol)]) , "\t\t" + str(Cor_sent), "\t\t" +  str(malformed), "\t\t"+ str(nonExistent) , "\t\t"+ str("%.2f" % SuccessRate) , "\t\t" + str("%.3f" %Presence))
    else:
        print(protocol, "\t\t" + str(protocol_count[protocols.index(protocol)]) , "\t\t" + str(Cor_sent), "\t\t" +  str(malformed), "\t\t"+ str(nonExistent) , "\t\t"+ str(SuccessRate) , "\t\t" + str("%.3f" %Presence))

print("\n" + str(total_count) + " Packets evaluated")



--------------------------------------- Fine-Tunning Results evaluation ---------------------------------------

Protocol    nº detections    Correctly sent    Malformed    Non-existent    %SuccessRate    %presence in total
IP 		396 		368 		28 		0 		92.93 		100.000
ICMP 		56 		56 		0 		0 		100.00 		14.141
DNS 		48 		48 		0 		0 		100.00 		12.121
UDP 		48 		48 		0 		0 		100.00 		12.121
ARP 		0 		0 		0 		0 		None 		0.000
HTTP 		52 		50 		2 		0 		96.15 		13.131
TCP 		264 		264 		0 		0 		100.00 		66.667

396 Packets evaluated
